# Packing and Relax
Keywords: PackRotamersMover, FastRelax, MoveMapFactory, cartesian, ResidueSelector, NeighborhoodResidueSelector, CDRResidueSelector, TaskFactory, TaskOperation, InitializeFromCommandline, RestrictToRepacking, PreventRepackingRLT, OperateOnResidueSubsetOperation, SimpleMetric, SequenceMetric, clone()

## Overview
Here, you will learn how to optimize the side-chains of a protein.  In Rosetta, we call this `packing`.  We will use TaskFactories to control which residues are optimized. We will use this knowledge to refine a specific region of a protein. 

**MOHINI KHEDEKAR**

In [ ]:
# You have to rerun this cell each time you start a new notebook or do a "factory reset".
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.mount_pyrosetta_install()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Notebook is set for PyRosetta use in Colab.  Have fun!


**Make sure you are in the directory with the pdb files:**


In [ ]:
cd /content/google_drive/MyDrive/codeschool2023

/content/google_drive/MyDrive/codeschool2023


## Imports

Before we begin, we must import some specific machinery from Rosetta.  Much of these tools are automatically imported when we do `from pyrosetta import *`, however, some are not. You should get into the habit of importing everything you need.  This will get you comfortable with the organization of Rosetta and make it easier to find tools that are beyond the scope of these workshops.

In [ ]:
#Python
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *

#Core Includes
from rosetta.core.kinematics import MoveMap
from rosetta.core.kinematics import FoldTree
from rosetta.core.pack.task import TaskFactory
from rosetta.core.pack.task import operation
from rosetta.core.simple_metrics import metrics
from rosetta.core.select import residue_selector as selections
from rosetta.core import select
from rosetta.core.select.movemap import *

#Protocol Includes
from rosetta.protocols import minimization_packing as pack_min
from rosetta.protocols import relax as rel
from rosetta.protocols.antibody.residue_selector import CDRResidueSelector
from rosetta.protocols.antibody import *
from rosetta.protocols.loops import *
from rosetta.protocols.relax import FastRelax


<ipython-input-3-139d7cba2c77>:7: UserWarning: Import of 'rosetta' as a top-level module is deprecated and may be removed in 2018, import via 'pyrosetta.rosetta'.
  from rosetta.core.kinematics import MoveMap


## Initialization 

Here, we will use command-line options to set the relax rounds to 2 instead of default 5 for speed of demo.  This is a bit tricky to do in code.
We also set the input antibody numbering scheme so that Rosetta understands the nomenclature of our antibody. 


Note that typically, we would add these options: `-ex1` and `-ex2` in order to increase the amount of rotamers
 available for packing, but this will slow us down for the demo, so we are keeping this out.

In [ ]:
init('-use_input_sc -input_ab_scheme AHo_Scheme -ignore_unrecognized_res \
     -ignore_zero_occupancy false -load_PDB_components false -relax:default_repeats 2 -no_fconfig')

PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2023.19+release.d7aa7f94e8be5e9d5110d37f167c2a7afd30c530 2023-05-08T16:22:16] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r348 2023.19+release.d7aa7f94e8b d7aa7f94e8be5e9d5110d37f167c2a7afd30c530 http://www.pyrosetta.org 2023-05-08T16:22:16
core.init: command: PyRosetta -use_input_sc -input_ab_scheme AHo_Scheme -ignore_unrecognized_res -ignore_zero_occupancy false -load_PDB_components false -relax:default_repeats 2 -no_fconfig -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1489914635 seed_offset=0 real_seed=-1489914635
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-1489914635 RG_type=mt19937


## Import and copy pose

Begin by importing a pose.  Here, we are going to use an antibody from the PDB. 
Note that we use the clone function to copy the pose into `original_pose`. Using the equal sign will only make what is known as a shallow copy in python and anything we do to pose will be seen in original_pose.  The clone operation copies all the data into the original_pose and is the Rosetta equivalent of the python module, `copy.deep_copy`. 

In [ ]:
#Import a pose
pose = pose_from_pdb("/content/google_drive/MyDrive/codeschool2023/2r0l_1_1.pdb")
original_pose = pose.clone()

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.228 seconds.
core.import_pose.import_pose: File '/content/google_drive/MyDrive/codeschool2023/2r0l_1_1.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ARG:CtermProteinFull 108
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue SER:CtermProteinFull 225
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ARG:CtermProteinFull 464
core.conformation.Conformation: Found disulfide between residues 23 88
core.conformation.Conformation: current variant for 23 CYS
core.conformation.Conformation: current variant for 88 CYS
core.conformation.Conformation: current variant for 23 CYD
core.conformation.Conformation: current variant for 88 CYD
core.conformation.Conformation: Found disulfide betw

## Setup a Normal TaskFactory

A `TaskFactory` is what we use to control packing specific residues in a pose. We first pass InitializeFromCommandLine which uses any of the options specified in the `pyrosetta.init()` function.

The `TaskFactory` is made up of a list of `TaskOperations`. These Taskops make up the bread and butter of controlling packing (and subsequently design). The taskops can be given to factory in any order.  

In Rosetta, ALL residues are set to both pack AND design by default. We use taskops to turn things off, like creating an ice sculpture.  Here, we turn design off by using the `RestrictToRepacking` operation.

In [ ]:
tf = TaskFactory()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(operation.RestrictToRepacking())

## Setup The Packer

Here, we setup the packer and pass the TaskFactory to it.  In general, if something can be packed, it will accept a TaskFactory.  Every time the protein is packed for a single round, the TaskFactory will generate what is called the PackerTask.  This object has all of the instructions needed for Rosetta to do packing and design.  Some TaskOperations can respond to environmental changes in the pose, such as neighboring residues and dynamically change at each packing step.  This is one of the reasons we use the TaskFactory machinery to setup packing instead of hacking the PackerTask itself as has been done in some of the earlier tutorials you may see on the web. 

In [ ]:
packer = pack_min.PackRotamersMover()
packer.task_factory(tf)

#Note that we are not passing a scorefunction here.  We will use the default, cmd-line scorefunction, 
# which is accessed through rosetta.core.scoring.get_score_function() and part of the packer.  We use use a scorefunction later. 

#Run the packer. (Note this may take a few minutes)
#Skip for tests
if not os.getenv("DEBUG"):
    packer.apply(pose)

#Dump the PDB
pose.dump_pdb('/content/google_drive/MyDrive/codeschool2023/2r0l_all_repack.pdb')

protocols.minimization_packing.PackRotamersMover: [ WARNING ] undefined ScoreFunction -- creating a default one
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions

True

Lets compare the energies of the before and after pose. Any difference?

In [ ]:
scorefxn = get_score_function()
before = scorefxn.score(original_pose)
after = scorefxn.score(pose)
print(before)
print(after)
#Finish the code here


core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
-520.7422105947901
-1036.400681555406


the energy for the pose after making changes decreases, which is good.

Check the sequence of the pose and the original pose using either the `SequenceMetric` 
 `rosetta.core.simple_metrics.metrics.SequenceMetric` (and it's calculate function) OR directly using the pose's `.sequence()` function. 
  Make sure that the poses have the same sequence - as packing is just design using single residue rotamers.

In [ ]:
sequence = rosetta.core.simple_metrics.metrics.SequenceMetric()
print(sequence.calculate(pose))
print(sequence.calculate(original_pose))

#original_pose.sequence()
#pose.sequence()



DIQMTQSPSSLSASVGDRVTITCRASQDVSTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQSYTTPPTFGQGTKVEIKREVQLVESGGGLVQPGGSLRLSCAASGFTISNSGIHWVRQAPGKGLEWVGWIYPTGGATDYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCARFWWRSFDYWGQGTLVTVSSIIGGSSSLPGSHPWLAAIYIGDSFCAGSLVHTCWVVSAAHCFSHSPPRDSVSVVLGQHFFNRTTDVTQTFGIEKYIPYTLYSVFNPSDHDLVLIRLKKKGDRCATRSQFVQPICLPEPGSTFPAGHKCQIAGWGHLDENVSGYSSSLREALVPLVADHKCSSPEVYGADISPNMLCAGYFDCKSDACQGDSGGPLACEKNGVAYLYGIISWGDGCGRLHKPGVYTRVANYVDWINDRIR
DIQMTQSPSSLSASVGDRVTITCRASQDVSTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQSYTTPPTFGQGTKVEIKREVQLVESGGGLVQPGGSLRLSCAASGFTISNSGIHWVRQAPGKGLEWVGWIYPTGGATDYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCARFWWRSFDYWGQGTLVTVSSIIGGSSSLPGSHPWLAAIYIGDSFCAGSLVHTCWVVSAAHCFSHSPPRDSVSVVLGQHFFNRTTDVTQTFGIEKYIPYTLYSVFNPSDHDLVLIRLKKKGDRCATRSQFVQPICLPEPGSTFPAGHKCQIAGWGHLDENVSGYSSSLREALVPLVADHKCSSPEVYGADISPNMLCAGYFDCKSDACQGDSGGPLACEKNGVAYLYGIISWGDGCGRLHKPGVYTRVANYVDWINDRIR


## Regional Packing

Lets pack just a single CDR loop.  How do we do this?  Use a `TaskFactory` and a `TaskOperation`.  
  To make things easier, we will use a `ResidueSelector` to select the CDR. You will see more of this later.
  Briefly, a ResidueSelector returns a boolean vector that is the length of the pose. Each boolean is an indication of whether the residue was selected or not.  Because the vector is the same length of the pose, the index is the residue number.  There are many `ResidueSelectors` in rosetta, including ones for `AND` and `OR` operations to combine them.  Note where most of them come from above: `from rosetta.core.select import residue_selector as selections`

### Make selection using the CDRResidueSelector and NeighborhoodResidueSelector

We will use the NeighborhoodResidueSelector to pack the CDR loop and its surrounding neighbors, which defaults to 6 angstroms.  Each time we pack, the TaskFactory will be used to generate packing instructions (`PackerTask`) - and subsequently, the neighbors that we are packing will be updated each time to reflect this changing state of the pose.

In [ ]:
nbr_selector = selections.NeighborhoodResidueSelector()
cdr_selector = CDRResidueSelector()
cdr_selector.set_cdr(h1)
#we are focusing on the cdr loop of the protein that is responsible for antibody identification

Note that h1 is what is called an enum.  It is a named integer.  This is better than passing around strings, 
 and you will start to see many of these around Rosetta. They were imported when we imported the antibody namespace at 
 the top of this workshop. Score terms such as fa_dun are also enums.

In [ ]:
nbr_selector.set_focus_selector(cdr_selector)
nbr_selector.set_include_focus_in_subset(True)

### Restrict to our selection

Lets turn off packing for everything but the H1 loop and its neighbors. By using a TF, every time packing is done,
  we regenerate the neighbors.  Using the NeighborhoodResidueSelector we actually use the pose energies object that
  has a list of neighbors instead of doing an N by N calculation each time!
  
In order to do this, we create what is known as a Residue Level Task Operation, or RLT, and then pass that into the `OperateOnResidueSubset`.  Use your tab completion to see how many RLTs there are.

In [ ]:
#True indicates here that we are flipping the selection.  So that we are turning off everything but the CDR and
#  its neighbors.

prevent_repacking_rlt = operation.PreventRepackingRLT()
prevent_subset_repacking = operation.OperateOnResidueSubset(prevent_repacking_rlt, nbr_selector, True )


Lets check to see what residues have been selected as the CDR, and then the CDR and its neighbors.  We will use this to make sure our PackerTask is setup properly

In [ ]:
cdr_res = []
print("CDR")
for i in select.get_residue_set_from_subset(cdr_selector.apply(pose)):
    print(i)
    cdr_res.append(i)
    
print("\nCDR+Neighbors")
for i in select.get_residue_set_from_subset(nbr_selector.apply(pose)):
    if i in cdr_res:
        print(i,"CDR")
    else:
        print(i)


CDR
basic.io.database: Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: Antibody CDR definition read successfully
antibody.AntibodyInfo: Successfully finished the CDR definition
antibody.AntibodyInfo: AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: ARFWWRSFDYW
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRClusterMatcher: Length: 10 Omega: TTTTTTTTTT
anti

### Reset the pose

In [ ]:

pose = original_pose.clone()

tf.push_back(prevent_subset_repacking)

pack_cdrs_and_neighbors_tf = tf.clone()

packer.task_factory(tf)

Before we start, lets take a look at our PackerTask.  Are we designing anything?  Does this match our selection?

In [ ]:
print(tf.create_task_and_apply_taskoperations(pose))

core.pack.task: Packer task: initialize from command line()
basic.io.database: Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: Antibody CDR definition read successfully
antibody.AntibodyInfo: Successfully finished the CDR definition
antibody.AntibodyInfo: AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: ARFWWRSFDYW
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H3
protocols.antibody.clus

Now lets Run the packer and dump the PDB.

Note how many rotamers were used and how many positions were done.  Now run it again.  Have those numbers changed?
 Why?

2943 rotamers were used at 60 positions. These have not changed after running it multiple times

In [ ]:
packer.apply(pose)

pose.dump_pdb("/content/google_drive/MyDrive/codeschool2023/CDRoutput.pdb")

core.pack.task: Packer task: initialize from command line()
basic.io.database: Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: Antibody CDR definition read successfully
antibody.AntibodyInfo: Successfully finished the CDR definition
antibody.AntibodyInfo: AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: ARFWWRSFDYW
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H3
protocols.antibody.clus

True

## Design

Lets design this CDR.   We already almost everything we need created, so lets do this!!

Note, since we added the RestrictToRepacking TaskOperation to our original `TaskFactory`, we need to reset (clear) it. 

In [ ]:

pose = original_pose.clone()

tf.clear()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(prevent_subset_repacking)

#Turn off design of neighbors
nbr_selector2 = selections.NeighborhoodResidueSelector()
nbr_selector2.set_focus_selector(cdr_selector)
nbr_selector2.set_include_focus_in_subset(False)

restrict_to_repack = operation.RestrictToRepackingRLT()
prevent_nbr_design = operation.OperateOnResidueSubset(restrict_to_repack, nbr_selector2, False )
tf.push_back(prevent_nbr_design)



Once again, lets check to make sure we have created the correct TaskFactory. You should now see that the CDRs we selected are set to design into any of the amino acids. 

In [ ]:
print(tf.create_task_and_apply_taskoperations(pose))

core.pack.task: Packer task: initialize from command line()
basic.io.database: Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: Antibody CDR definition read successfully
antibody.AntibodyInfo: Successfully finished the CDR definition
antibody.AntibodyInfo: AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: ARFWWRSFDYW
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H3
protocols.antibody.clus

### Set and Run

Set the new tf to the packer and run it. This will take a few minutes as for each CDR we packing rotamers for all 20 amino acids instead of just the native residue. Take note of how many rotamers are built compared to when we didn't do any design.

Dump the PDB and take a look. 

In [ ]:
packer = pack_min.PackRotamersMover()
packer.task_factory(tf)

#Note that we are not passing a scorefunction here.  We will use the default, cmd-line scorefunction, 
# which is accessed through rosetta.core.scoring.get_score_function() and part of the packer.  We use use a scorefunction later. 

#Run the packer. (Note this may take a few minutes)
#Skip for tests
if not os.getenv("DEBUG"):
    packer.apply(pose)

#Dump the PDB
pose.dump_pdb('/content/google_drive/MyDrive/codeschool2023/new2r0l_all_repack.pdb')

protocols.minimization_packing.PackRotamersMover: [ WARNING ] undefined ScoreFunction -- creating a default one
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.task: Packer task: initialize from command line()
basic.io.database: Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: Antibody CDR definition read successfully
antibody.AntibodyInfo: Successfully finished the CDR definition
antibody.AntibodyInfo: AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: ARFWWRSFDYW
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: Setting up 

True

## Relax

Lets first relax the whole protein.  This will take some time, so after you run it, take a break and stretch!  As usual, you can find the output in the `expected_outputs` directory of this directory.


In [ ]:

pose = original_pose.clone()
fr = FastRelax()

#Here, we have to set a scorefunction or we segfault.  
#  Error checking is important, and protocols should use a default scorefunction. We will manage.

fr.set_scorefxn(scorefxn)

#Lets run this.  This takes a very long time, so we are going decrease the amount of minimization cycles we use.
# This is generally only recommended for cartesian, but we just want everything to run fast at the moment.
fr.max_iter(100)

#Run the code
fr.apply(pose)
pose.dump_pdb("/content/google_drive/MyDrive/codeschool2023/relaxedprotein.pdb")

#Dump the pdb and take a look.



core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.relax.FastRelax: CMD: repeat  -520.742  0  0  0.55
protocols.relax.FastRelax: CMD: coord_cst_weight  -520.742  0  0  0.55
protocols.relax.FastRelax: CMD: scale:fa_rep  -895.085  0  0  0.022
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 7459 rotamers at 464 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -1484.42  0  0  0.022
protocols.relax.FastRelax: CMD: scale:fa_rep  -1470.09  0  0  0.02805
core.optimization.Minimizer: [ WARNING ] LBFGS MAX CYCLES 100 EXCEEDED, BUT FUNC NOT CONVERGED!
protocols.relax.FastRelax: CMD: min  -1963.44  1.62717  1.62717  0.02805
protocols.relax.FastRelax: CMD: coord_cst_weight  -1963.44  1.62717  1.62717  0.02805
protocols.relax.FastRelax: CMD: scale:fa_rep  -1287.31  1.62717  1.62717  0.14575
core.pack.task: Packer task: initialize from command line

True

Check the energy difference here once again.  Note how low it is compared to simply packing

delta = ?

In [ ]:
'''
relaxed = scorefxn(pose)
original = scorefxn(original_pose)
DE = relaxed - original
print(DE)'''

scorefxn = get_score_function()
original = scorefxn.score(original_pose)
relaxed = scorefxn.score(pose)
DE = relaxed - original
print(DE)

'''
print(before)
print(after)'''



core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
-977.0181611447786


'\nprint(before)\nprint(after)'

### Regional Relax

So that was fun, but what we really want is to optimize regions of the protein that we care about. 
  This can be tricky, so to start off, lets do the basic thing and see what happens. 
  
We will use a `MoveMapFactory`, which allows us to use `ResidueSelectors`.  
More information on the `MoveMapFactory` in terms of RosettaScripts, can be found here: https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/MoveMapFactories/MoveMapFactories-RosettaScripts

The default is to have everything OFF first, and turn specific things on.

In [ ]:

pose = original_pose.clone()
mmf = MoveMapFactory()

#mm_enable and mm_disable are Enums (numbered variables) that come when we import the MMF
mmf.add_bb_action(mm_enable, cdr_selector)
#mmf.add_chi_action(mm_enable, cdr_selector) We are taking this out for speed.

mm  = mmf.create_movemap_from_pose(pose)

basic.io.database: Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: Antibody CDR definition read successfully
antibody.AntibodyInfo: Successfully finished the CDR definition
antibody.AntibodyInfo: AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: ARFWWRSFDYW
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRClusterMatcher: Length: 10 Omega: TTTTTTTTTT
antibody

Lets take a look at our `MoveMap` that the `MoveMapFactory` creates. Pass the movemap to `print()`  We should only have a few residues there.

Here, we only want to pack the region and neighbors.  NOT all 500 residues!

Lets use the cloned TaskFactory we used to repack the CDRs and neighbors, since we will be allowing the backbone of the CDRs to move.

In [ ]:
fr.set_movemap_factory(mmf)
fr.set_task_factory(pack_cdrs_and_neighbors_tf)

Run this, take a break and then lets see the results.
Dump the pose and take a look at it.  

In [ ]:
if not os.getenv("DEBUG"):
    fr.apply(pose)
    
pose.dump_pdb("/content/google_drive/MyDrive/codeschool2023/regionalrelaxpdb")

basic.io.database: Database file opened: sampling/antibodies/cluster_center_dihedrals.txt
protocols.antibody.AntibodyNumberingParser: Antibody numbering scheme definitions read successfully
protocols.antibody.AntibodyNumberingParser: Antibody CDR definition read successfully
antibody.AntibodyInfo: Successfully finished the CDR definition
antibody.AntibodyInfo: AC Detecting Regular CDR H3 Stem Type
antibody.AntibodyInfo: ARFWWRSFDYW
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: KINKED
antibody.AntibodyInfo: AC Finished Detecting Regular CDR H3 Stem Type: Kink: 1 Extended: 0
antibody.AntibodyInfo: Setting up CDR Cluster for H1
protocols.antibody.cluster.CDRClusterMatcher: Length: 13 Omega: TTTTTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H2
protocols.antibody.cluster.CDRClusterMatcher: Length: 10 Omega: TTTTTTTTTT
antibody.AntibodyInfo: Setting up CDR Cluster for H3
protocols.antibody.cluster.CDRClusterMatcher: Length: 10 Omega: TTTTTTTTTT
antibody

True

**What is wrong with this Pose???**


Okay, so we know whats wrong here, right?  Right?
The pose is moving too much where we don't want it. This is due to the FoldTree
  There are two fixes for this - modify the foldtree (using a Loop foldtree) or use cartesian-space refinement. 
  
  Lets do the easier of the two - cartesian!
  
 ### Regional Relax - Cartesian

In [ ]:

pose = original_pose.clone()
cart_sf = create_score_function("ref2015_cart")
mmf.set_cartesian(True)
fr.set_movemap_factory(mmf)
fr.set_scorefxn(cart_sf)
fr.cartesian(True)

#This is a general recommendation for cartesian minimization - it lowers the number of maximum cycles.
# More than this only increases time of protocol, but has little effect on energies/structure
fr.max_iter(200)

Run relax Lets dump the pose and take a look again.  Does this look better?   How are the energies?

In [ ]:

# YOUR CODE HERE
raise NotImplementedError()

## Conclusions

That should get you started with packing/relax! 
